# getting the data, understanding the data :

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/voicegender/voice.csv')

In [ ]:
df.head()

In [ ]:
df.describe().transpose()

In [ ]:
l = df.columns

In [ ]:
df.isnull().sum()

In [ ]:
def encode(x):
    if x == 'male':
        return 1
    else:
        return 0

In [ ]:
df['label'].unique()

In [ ]:
df['label'] = df['label'].apply(encode)

# exploratory data analysis :

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns 

In [ ]:
plt.figure(figsize=(16,8))
sns.kdeplot(df['meanfreq'])

In [ ]:
sns.countplot(df['label'])

*well balanced dataset not skewed towards either of the gender !*

# using sk-learn for classification :

**imports from sklearn:**

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression

from sklearn.svm import SVC

from sklearn.naive_bayes import MultinomialNB

from sklearn.ensemble import RandomForestClassifier

from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df[l[:-1]],df[l[-1]],random_state=101,test_size=0.33)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

scaled_train = scaler.fit_transform(X_train)
scaled_test = scaler.transform(X_test)

**logistic regression:**

In [ ]:
logistic_model = LogisticRegression()

In [ ]:
logistic_model.fit(scaled_train,y_train)

In [ ]:
preds = logistic_model.predict(scaled_test)

In [ ]:
print(classification_report(y_test,preds))

In [ ]:
print(confusion_matrix(y_test,preds))

**using PCA, SVC :**

In [ ]:
pca = PCA(n_components=10)

In [ ]:
reduced_train = pca.fit_transform(scaled_train)
reduced_test = pca.transform(scaled_test)

In [ ]:
svc = SVC()

In [ ]:
svc.fit(reduced_train,y_train)

In [ ]:
preds = svc.predict(reduced_test)

In [ ]:
print(classification_report(y_test,preds))

In [ ]:
print(confusion_matrix(y_test,preds))

*as we see dropping some features increases the accuracy, we set up a loop to find the best number of features :*

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy = []
i_vals = []
for i in range(1,21):
    
    pca = PCA(n_components=i)
    reduced_train = pca.fit_transform(scaled_train)
    reduced_test = pca.transform(scaled_test)
    
    svc = SVC()
    svc.fit(reduced_train,y_train)
    pred = svc.predict(reduced_test)
    
    accuracy.append(accuracy_score(y_test,pred))
    i_vals.append(i)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(i_vals, accuracy)

In [ ]:
i_vals = np.array(i_vals).reshape(20,1)
accuracy = np.array(accuracy).reshape(20,1)


In [ ]:
np.argmax(accuracy)

**using the above i prove the verification for choosing 10 dimensions !**

**multinomial naive bayes:**

In [ ]:
mnb = MultinomialNB()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()

scaled_nn_train = mms.fit_transform(X_train)
scaled_nn_test = mms.transform(X_test)

mnb.fit(scaled_nn_train,y_train)
preds = mnb.predict(scaled_nn_test)



In [ ]:
print(classification_report(y_test,preds))

**using random forest classifier:**

In [ ]:
i_vals = []
accu = [] 
for i in range(1,200):
    
    random_forest = RandomForestClassifier(n_estimators=i)
    random_forest.fit(scaled_nn_train,y_train)
    
    preds = random_forest.predict(scaled_nn_test)
    
    i_vals.append(i)
    accu.append(accuracy_score(y_test,preds))

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(i_vals,accu)

In [ ]:
i_vals = np.array(i_vals)

accu = np.array(accu)

In [ ]:
i_vals[np.argmax(accu)]

In [ ]:
random_forest = RandomForestClassifier(n_estimators=134)
random_forest.fit(scaled_nn_train,y_train)
    
preds = random_forest.predict(scaled_nn_test)

In [ ]:
print(classification_report(y_test,preds))

**constructing neural networks to classify:**

In [ ]:
from tensorflow.keras.layers import Dense

from tensorflow.keras.models import Sequential

from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()

model.add(Dense(10,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(1,activation='sigmoid'))


model.compile(optimizer = 'adam', metrics = ['acc'], loss='binary_crossentropy')
call = [EarlyStopping(monitor='val_acc', patience=25)]

In [ ]:
model.fit(reduced_train, y_train, validation_data=(reduced_test,y_test), epochs=700, callbacks=call)

In [ ]:
plt.figure(figsize=(16,6))
pd.DataFrame(model.history.history).plot()

In [ ]:
preds = model.predict_classes(reduced_test)

In [ ]:
print(classification_report(y_test,preds))